Printeo del nuevo modelo usado

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import _init_paths

import os

import torch
import torch.utils.data
from opts import opts
from models.model import create_model, load_model, save_model
from models.data_parallel import DataParallel
from logger import Logger
from datasets.dataset_factory import get_dataset
from trains.train_factory import train_factory
import cv2
import sys
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
sys.argv = ['-f', 'fewdetv3', '--exp_id', 'coco_dlav7','--num_workers', '0','--batch_size','2', '--arch','dlav7_34'] #cmd = 'ctdet --exp_id coco_dla --batch_size 4 --lr 1.25e-4  --gpus 0 --num_workers 0'
opt = opts().parse()
print(opt.task)
print(opt.num_workers)
print(opt.batch_size)

Fix size testing.
training chunk_sizes: [2]
The output will be saved to  C:\Users\avata\FewShot-CenterNet\src\lib\..\..\exp\fewdetv3\coco_dlav7
fewdetv3
0
2


In [3]:
torch.manual_seed(opt.seed)
torch.backends.cudnn.benchmark = not opt.not_cuda_benchmark and not opt.test

In [4]:
opt.reg_loss

'l1'

In [5]:
Dataset = get_dataset(opt.dataset, opt.task)
opt = opts().update_dataset_info_and_set_heads(opt, Dataset)
logger = Logger(opt)
os.environ['CUDA_VISIBLE_DEVICES'] = opt.gpus_str
opt.device = torch.device('cuda' if opt.gpus[0] >= 0 else 'cpu')

heads {'ss': 80, 'hm': 1, 'wh': 2, 'reg': 2}


In [6]:
model = create_model(opt.arch, opt.heads, opt.head_conv)
optimizer = torch.optim.Adam(model.parameters(), opt.lr)
start_epoch = 0
if opt.load_model != '':
    model, optimizer, start_epoch = load_model(model, opt.load_model, optimizer, opt.resume, opt.lr, opt.lr_step)

In [7]:
Trainer = train_factory[opt.task]
trainer = Trainer(opt, model, optimizer)
trainer.set_device(opt.gpus, opt.chunk_sizes, opt.device)

In [8]:
train_loader = torch.utils.data.DataLoader(
      Dataset(opt, 'train'), 
      batch_size=opt.batch_size, 
      shuffle=True,
      num_workers=opt.num_workers,
      pin_memory=True,
      drop_last=True
  )

==> initializing coco 2017 train data.
loading annotations into memory...
Done (t=15.15s)
creating index...
index created!
Loaded train 118287 samples


In [9]:
img = next(iter(train_loader))

In [10]:
print(img.keys())
print(img['input'].size())

dict_keys(['input', 'ss', 'hm', 'reg_mask', 'ind', 'wh', 'reg'])
torch.Size([2, 3, 512, 512])


In [11]:
model.to('cuda')
torch.cuda.empty_cache()
outputs = model(img['input'].to('cuda'))

In [12]:
output = outputs[0]
print(output.keys())

dict_keys(['fc_ss', 'ss', 'hm', 'wh', 'reg'])


In [13]:
from models.losses import *
from models.utils import _transpose_and_gather_feat

In [14]:
batch=img
mask = batch['reg_mask']
ind = batch['ind']
target_wh = batch['wh']
target_reg = batch['reg']
target_ss = batch['ss']

In [15]:
target_wh.size()

torch.Size([2, 128, 2])

In [18]:
target_ss.size()

torch.Size([2, 128])

In [19]:
print(output['wh'].size())
pred =_transpose_and_gather_feat(output['wh'].to('cuda'),ind.to('cuda'))
print(pred.size())
print(mask.size())
mask2 = mask.unsqueeze(2).expand_as(pred).float()
print(mask2.size())

torch.Size([2, 2, 128, 128])
torch.Size([2, 128, 2])
torch.Size([2, 128])
torch.Size([2, 128, 2])


In [20]:
print(output['ss'].size())
pred_ss =_transpose_and_gather_feat(output['ss'].to('cuda'),ind.to('cuda'))
print(pred_ss.size())
print(mask.size())
mask3 = mask.unsqueeze(2).expand_as(pred_ss).float()
print(mask3.size())

torch.Size([2, 80, 128, 128])
torch.Size([2, 128, 80])
torch.Size([2, 128])
torch.Size([2, 128, 80])


In [26]:
target_ss[mask.bool()]

tensor([33, 62, 15])

In [36]:
pred_ss[mask.bool()]

tensor([[0.0126, 0.0089, 0.0007, 0.0082, 0.0046, 0.0070, 0.0228, 0.0116, 0.0320,
         0.0285, 0.0031, 0.0230, 0.0361, 0.0120, 0.0047, 0.0089, 0.0011, 0.0016,
         0.0106, 0.0062, 0.0118, 0.0309, 0.0055, 0.0017, 0.0083, 0.0035, 0.0025,
         0.0005, 0.0425, 0.0030, 0.0011, 0.0066, 0.0215, 0.0141, 0.0808, 0.0622,
         0.0040, 0.0080, 0.0235, 0.0093, 0.0113, 0.0077, 0.0014, 0.0432, 0.0036,
         0.0024, 0.0053, 0.0025, 0.0053, 0.0036, 0.0072, 0.0095, 0.0303, 0.0056,
         0.0055, 0.0100, 0.0107, 0.0692, 0.0028, 0.0031, 0.0021, 0.0017, 0.0082,
         0.0043, 0.0202, 0.0102, 0.0013, 0.0027, 0.0175, 0.0172, 0.0328, 0.0026,
         0.0063, 0.0088, 0.0025, 0.0049, 0.0107, 0.0045, 0.0121, 0.0032],
        [0.0281, 0.0089, 0.0050, 0.0107, 0.0253, 0.0961, 0.0043, 0.0071, 0.0237,
         0.0047, 0.0032, 0.0206, 0.0083, 0.0077, 0.0086, 0.0050, 0.0010, 0.0077,
         0.0046, 0.0155, 0.0060, 0.0048, 0.0012, 0.0095, 0.0030, 0.0017, 0.0196,
         0.0755, 0.0082, 0.0860, 0.

In [39]:
import torch.nn as nn

In [46]:
nn.CrossEntropyLoss()(pred_ss[mask.bool()].to('cuda'),target_ss[mask.bool()].to('cuda'))

tensor(4.3819, device='cuda:0', grad_fn=<NllLossBackward>)

In [49]:
F.cross_entropy(pred_ss[mask.bool()].to('cuda'),target_ss[mask.bool()].to('cuda'))

tensor(4.3819, device='cuda:0', grad_fn=<NllLossBackward>)

# Respecto a la deteccion

In [ ]:
print()